In [ ]:
%pip install torch torchvision torchaudio
%pip install transformers
%pip install pandas sklearn
%pip install ipywidgets
%pip install --upgrade notebook
%pip install scikit-learn

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch

df = pd.read_csv('preprocessing/preprocessed_data.csv', dtype={'id': str}, encoding='latin-1')

df['article'] = df['article'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
df['article'] = df['article'].astype(str)

df.sentiments.value_counts()

sentiments
positive    4567
neutral     3346
negative    2948
Name: count, dtype: int64

In [2]:
df[:4]

,topic,title,source,article_link,article,sentiments,processed_article
0,Microsoft,The Top 7 Tech Stocks to Buy in March 2024,InvestorPlace,https://investorplace.com/2024/03/the-top-7-te...,Microsoft (MSFT) is still growing at a fast pa...,positive,"[[101, 7513, 1006, 5796, 6199, 1007, 2003, 214..."
1,Microsoft,Microsoft-owned LinkedIn makes this 'first-eve...,Times of India,https://timesofindia.indiatimes.com/gadgets-ne...,Microsoft-owned LinkedIn generated $1.7 billio...,positive,"[[101, 7513, 1011, 3079, 5799, 2378, 7013, 100..."
2,Microsoft,Microsoft debuts Copilot for finance pros,Computerworld,https://www.computerworld.com/article/3714142/...,Microsoft has launched a Copilot assistant for...,neutral,"[[101, 7513, 2038, 3390, 1037, 8872, 22360, 21..."
3,Microsoft,Microsoft's Path to Becoming the Largest Compa...,The Motley Fool,https://www.fool.com/investing/2024/03/06/micr...,Microsoft's Path to Becoming the Largest Compa...,positive,"[[101, 7513, 1005, 1055, 4130, 2000, 3352, 199..."


In [3]:
texts = df['article'].values
df['sentiments'] = df['sentiments'].map({'positive': 2, 'neutral': 1, 'negative': 0})

In [4]:
X_train,X_test,y_train,y_test = train_test_split(df.index.values,
                                                df.sentiments.values,
                                                test_size = 0.15,
                                                random_state=42)

# Step 1: Reconstruct the Test DataFrame from indices
test_set_df = df.loc[X_test]

# Step 2: Add Test Labels to the DataFrame
# Ensure that the index of y_test aligns with test_set_df; reset index if needed
test_set_df = test_set_df.reset_index(drop=True)
test_set_df['sentiments'] = y_test

# Step 3: Save the DataFrame to a CSV file
test_set_df.to_csv('test_set.csv', index=False)

df.loc[X_train,'type'] = 'train'
df.loc[X_test,'type'] = 'test'

In [5]:
df.groupby(['sentiments','type']).count()

topic  title  source  article_link  article  \
sentiments type                                                 
0          test     457    457     457           457      457   
           train   2491   2491    2491          2491     2491   
1          test     513    513     513           513      513   
           train   2833   2833    2833          2833     2833   
2          test     660    660     660           660      660   
           train   3907   3907    3907          3907     3907   

                  processed_article  
sentiments type                      
0          test                 457  
           train               2491  
1          test                 513  
           train               2832  
2          test                 660  
           train               3907

In [6]:
from torch.utils.data import TensorDataset
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

encoder_train = tokenizer.batch_encode_plus(df[df["type"]=='train'].article.values,
                                           add_special_tokens = True,
                                            return_attention_mask = True,
                                           padding='max_length',
                                           max_length = 512,
                                           truncation=True,
                                           return_tensors = 'pt')



encoder_test = tokenizer.batch_encode_plus(df[df["type"]=='test'].article.values,
                                           add_special_tokens = True,
                                            return_attention_mask = True,
                                           padding='max_length',
                                           max_length = 512,
                                           truncation=True,
                                           return_tensors = 'pt')

input_ids_train = encoder_train['input_ids']
attention_masks_train = encoder_train["attention_mask"]
labels_train = torch.tensor(df[df['type']=='train'].sentiments.values)


input_ids_test = encoder_test['input_ids']
attention_masks_test = encoder_test["attention_mask"]
labels_test = torch.tensor(df[df['type']=='test'].sentiments.values)

In [7]:
data_train = TensorDataset(input_ids_train,attention_masks_train,labels_train)
data_test = TensorDataset(input_ids_test,attention_masks_test,labels_test)

In [10]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                     num_labels = 3,
                                     output_attentions = False,
                                     output_hidden_states =  False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from torch.utils.data import RandomSampler,SequentialSampler,DataLoader

dataloader_train = DataLoader(
    data_train,
    sampler= RandomSampler(data_train),
    batch_size = 16
    
)


dataloader_val = DataLoader(
    data_test,
    sampler= RandomSampler(data_test),
    batch_size = 32
    
)

In [12]:
from transformers import AdamW,get_linear_schedule_with_warmup
optimizer = AdamW(model.parameters(),lr = 1e-5,eps = 1e-8)

epochs  = 10
scheduler = get_linear_schedule_with_warmup(
            optimizer,
    num_warmup_steps = 0,
   num_training_steps = len(dataloader_train)*epochs 
)

/Users/xavier/.pyenv/versions/3.8.18/lib/python3.8/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f"Loading:{device}")

Loading:cpu


In [15]:
from tqdm.notebook import tqdm

def evaluate(dataloader_val):
    model.eval()
    
    loss_val_total = 0
    predictions,true_vals = [],[]
    
    for batch in tqdm(dataloader_val):
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':  batch[0],
                  'attention_mask':batch[1],
                  'labels': batch[2]
                 }
        with torch.no_grad():
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total +=loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
        
        
    loss_val_avg = loss_val_total/len(dataloader_val)  
    
    predictions = np.concatenate(predictions,axis=0)
    true_vals = np.concatenate(true_vals,axis=0) 
    return loss_val_avg,predictions,true_vals

In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score

def f1_score_func(preds,labels):
    preds_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat,preds_flat,average = 'weighted')

for epoch in tqdm(range(1,epochs+1)):
    model.train()
    
    loss_train_total=0
    
    progress_bar = tqdm(dataloader_train,desc = "Epoch: {:1d}".format(epoch),leave = False,disable = False)
    
    
    for batch in progress_bar:
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            "input_ids":batch[0],
            "attention_mask":batch[1],
            "labels":batch[2]
            
        }
        outputs = model(**inputs)
        
        loss = outputs[0]
#         logits = outputs[1]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        
        optimizer.step()
        scheduler.step()
        
        
        progress_bar.set_postfix({'training_loss':'{:.3f}'.format(loss.item()/len(batch))})
#     torch.save(model.state_dict(),f'/kaggle/output/BERT_ft_epoch{epoch}.model')To save the model after each epoch
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training Loss: {loss_train_avg}')
    val_loss,predictions,true_vals = evaluate(dataloader_test)
    test_score = f1_score_func(predictions,true_vals)
    tqdm.write(f'Val Loss:{val_loss}\n Test Score:{test_score}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1:   0%|          | 0/577 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
probabilities = torch.softmax(torch.tensor(predictions), dim=1).numpy()
# Convert probabilities to predicted class (highest probability)
predicted_classes = np.argmax(probabilities, axis=1)

# Calculate Precision and Recall
precision = precision_score(true_vals, predicted_classes, average='macro')
recall = recall_score(true_vals, predicted_classes, average='macro')

print(f"Precision: {precision}")
print(f"Recall: {recall}")

In [17]:
def precision_recall_f1(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    precision = precision_score(labels_flat, preds_flat, average='weighted')
    recall = recall_score(labels_flat, preds_flat, average='weighted')
    f1 = f1_score(labels_flat, preds_flat, average='weighted')
    return precision, recall, f1

# After each epoch or at the end of your training, calculate additional metrics
val_loss, predictions, true_vals = evaluate(dataloader_test)
precision, recall, f1 = precision_recall_f1(predictions, true_vals)

tqdm.write(f'Val Loss: {val_loss}\n Precision: {precision}\n Recall: {recall}\n F1 Score: {f1}')

  0%|          | 0/51 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
torch.save(model, 'model_full.pth')